<a href="https://colab.research.google.com/github/Matthew0803/Machine-Learning/blob/main/Google%20Colab/ai_flower.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install tensorflow

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np
import matplotlib.pyplot as plt
import os
import pathlib
# 1. Load Data
# For example, loading the flower photos from the TensorFlow dataset:
data_dir = tf.keras.utils.get_file('flower_photos', origin='https://storage.googleapis.com/download.tensorflow.org/example_images/flower_photos.tgz', untar=True)
data_dir = pathlib.Path(data_dir)
image_count = len(list(data_dir.glob('*/*.jpg')))
print(f"Total number of images: {image_count}")
batch_size = 32
img_height = 180
img_width = 180
image_size = (img_height, img_width)


train_ds = tf.keras.utils.image_dataset_from_directory(
    data_dir,
    validation_split=0.2,
    subset="training",
    seed=123,
    image_size=image_size,
    batch_size=batch_size
)

val_ds = tf.keras.utils.image_dataset_from_directory(
    data_dir,
    validation_split=0.2,
    subset="validation",
    seed=123,
    image_size=image_size,
    batch_size=batch_size
)

class_names = train_ds.class_names
num_classes = len(class_names)
print(f"Class names: {class_names}")

228813984/228813984 ━━━━━━━━━━━━━━━━━━━━ 3s 0us/step
Total number of images: 3670
Found 3670 files belonging to 5 classes.
Using 2936 files for training.
Found 3670 files belonging to 5 classes.
Using 734 files for validation.
Class names: ['daisy', 'dandelion', 'roses', 'sunflowers', 'tulips']


In [ ]:
# 2. Preprocess Data
def preprocess(image, label):
    image = tf.image.convert_image_dtype(image, tf.float32)
    return image, label

train_ds = train_ds.map(preprocess)
val_ds = val_ds.map(preprocess)


AUTOTUNE = tf.data.AUTOTUNE
train_ds = train_ds.cache().prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

In [ ]:
# 3. Create Model
model = keras.Sequential([
    layers.Conv2D(32, 3, activation='relu', input_shape=(img_height, img_width, 3)),
    layers.MaxPooling2D(),
    layers.Conv2D(64, 3, activation='relu'),
    layers.MaxPooling2D(),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(num_classes) # number of classes for the dataset

])

/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [ ]:
# 4. Compile Model
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [ ]:
# 5. Train Model
epochs = 10
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=epochs
)

Epoch 1/10
92/92 ━━━━━━━━━━━━━━━━━━━━ 212s 2s/step - accuracy: 0.2566 - loss: 337.9529 - val_accuracy: 0.3569 - val_loss: 1.5259
Epoch 2/10
92/92 ━━━━━━━━━━━━━━━━━━━━ 268s 2s/step - accuracy: 0.4696 - loss: 1.3860 - val_accuracy: 0.3011 - val_loss: 1.6836
Epoch 3/10
92/92 ━━━━━━━━━━━━━━━━━━━━ 208s 2s/step - accuracy: 0.6388 - loss: 1.0544 - val_accuracy: 0.3433 - val_loss: 2.1104
Epoch 4/10
92/92 ━━━━━━━━━━━━━━━━━━━━ 226s 2s/step - accuracy: 0.7811 - loss: 0.6719 - val_accuracy: 0.3665 - val_loss: 2.5927
Epoch 5/10
92/92 ━━━━━━━━━━━━━━━━━━━━ 208s 2s/step - accuracy: 0.8752 - loss: 0.4417 - val_accuracy: 0.3529 - val_loss: 2.9130
Epoch 6/10
92/92 ━━━━━━━━━━━━━━━━━━━━ 272s 2s/step - accuracy: 0.8648 - loss: 0.4832 - val_accuracy: 0.3597 - val_loss: 3.2551
Epoch 7/10
92/92 ━━━━━━━━━━━━━━━━━━━━ 266s 2s/step - accuracy: 0.9074 - loss: 0.3601 - val_accuracy: 0.3529 - val_loss: 4.5095
Epoch 8/10
92/92 ━━━━━━━━━━━━━━━━━━━━ 250s 2s/step - accuracy: 0.9061 - loss: 0.3092 - val_accuracy: 0.3215 -

In [ ]:
# 6. Evaluate Model
loss, accuracy = model.evaluate(val_ds)
print(f'Test Loss: {loss:.4f}')
print(f'Test Accuracy: {accuracy:.4f}')

23/23 ━━━━━━━━━━━━━━━━━━━━ 17s 689ms/step - accuracy: 0.3719 - loss: 5.3597
Test Loss: 5.8131
Test Accuracy: 0.3706


In [ ]:
# 7. Predict on new data
image_path = str(data_dir) + '/roses/10503217854_e66a804309.jpg' # example path, pick one from the test
img = tf.keras.utils.load_img(image_path, target_size = image_size)
img_array = tf.keras.utils.img_to_array(img)
img_array = tf.expand_dims(img_array, 0) # Create a batch
img_array = tf.image.convert_image_dtype(img_array, dtype=tf.float32)
predictions = model.predict(img_array)
score = tf.nn.softmax(predictions[0])
print(f"Predicted flower is {class_names[np.argmax(score)]} with a confidence of {100 * np.max(score):.2f} %")

# Optional: Save model
# model.save('flower_detector_model')

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
Predicted flower is roses with a confidence of 100.00 %
